In [11]:
import itertools
import pickle
import pandas as pd
import numpy as np
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor
#allow autocompletion in notebook
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import sys
sys.path.append("/Users/bourn/OneDrive/Documents/CalcAMP/")

from CalcAMP import calcamp

## Import the model you want to use

In [14]:
#give the path of the model you want to use from the folder Model

path_mdl = "/Users/bourn/OneDrive/Documents/CalcAMP/Model/RF_Fungi_feat_sel.sav"


calcamp.LoadModel("/Users/bourn/OneDrive/Documents/CalcAMP/Model/RF_Fungi_feat_sel.sav")

C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=25,
                       min_samples_leaf=4)

In [5]:
#calculate all descriptors 
#Pybiomed descriptors
#AA composition (20), Di-peptide compostion (400), CTD descriptors (147)
#Create a specific df for each and then concatenate them 

def calculate_all_desc(df):
    aacomp=[]
    ctdcomp=[]
    dpcomp=[]
    paacomp=[]

    for seq in df.Sequence:
        protein_class = PyProtein.PyProtein(seq)
        aacomp.append(protein_class.GetAAComp()) #aa composition
        ctdcomp.append(protein_class.GetCTD()) #CTD descriptors (147 features)
        dpcomp.append(protein_class.GetDPComp()) #Di peptide descriptors (400 features)
        paacomp.append(protein_class.GetPAAC(lamda=4,weight=0.05)) #Pseudoamino acid composition (24)

    df_aa = pd.DataFrame(aacomp)
    df_ctd = pd.DataFrame(ctdcomp)
    df_dpc = pd.DataFrame(dpcomp)
    df_dpc.rename(columns={'MW': 'MW_dipep'}, inplace=True) #to avoid two columns with the same name
    df_paa = pd.DataFrame(paacomp)

    #ModelAMP physicochemical descriptors (10)
    B = GlobalDescriptor(list(df.Sequence))
    B.calculate_all()
    df_desc = pd.DataFrame(B.descriptor, columns=B.featurenames)


    df_desc_all = pd.concat([df_aa, df_ctd, df_dpc, df_desc, df_paa], axis=1)
    return df_desc_all

def get_pred(L_pep): 
    df_seq = pd.DataFrame()
    df_seq['Sequence'] = L_pep
    df_seq_desc = calculate_all_desc(df_seq)
    g_pos_prob = et_tuned_g_pos.predict_proba(df_seq_desc)
    g_neg_prob = lgbm_tuned_g_neg.predict_proba(df_seq_desc)

    df_seq['Gram+_pred_non_amp'] = [np.round(i[0], 3) for i in g_pos_prob]
    df_seq['Gram+_pred_amp'] = [np.round(i[1], 3) for i in g_pos_prob]
    df_seq['Gram-_pred_non_amp'] = [np.round(i[0], 3) for i in g_neg_prob]
    df_seq['Gram-_pred_amp'] = [np.round(i[1], 3) for i in g_neg_prob]
    df_seq['Gram+'] = et_tuned_g_pos.predict(df_seq_desc)
    df_seq['Gram-'] = lgbm_tuned_g_neg.predict(df_seq_desc)
    return df_seq

In [19]:
#get two best model Gram+ and Gram-
et_tuned_g_pos = pickle.load(open(r'D:Colin/amp_paper/model/ET_gram_pos_full.sav', 'rb'))
lgbm_tuned_g_neg = pickle.load(open(r'D:Colin/amp_paper/model/LGBM_gram_neg_full.sav', 'rb'))

C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator ExtraTreeClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator ExtraTreesClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 1.0.2. This migh

## From FASTA file 

In [27]:
fasta_file = r'D:Colin/amp_paper/dataset_in_house_CD_hit_60.fasta' #replace with the path of your file
df = calcamp.OpenFasta(fasta_file)
#calcamp.CalculateAllDescriptors(df)
df

,ID,Sequence,Length
0,2,FKCRRWQWRMAKLGA,15
1,3,PKIISSPLFKTLLSAVGSALSSSGGQE,27
2,9,TLKKSLLLIFFLGTISLSLC,20
3,15,NGVQPKYRWWRWWRRWWW,18
4,17,ITEVITILLNRLTDRLEK,18
...,...,...,...
337,1866,GCKKYRRFRWKFKGKLFLFG,20
338,1878,SAVWRHWRRFWLRKHRKH,18
339,1886,FLFKLIPKAIKGLVKAIRK,19
340,1889,GRLQAFLAKMKEIAAQTL,18


In [32]:
calcamp.GetAllPred(df.Sequence, et_tuned_g_pos)

C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


,Sequence,Proba_pred_non_amp,Proba_pred_amp,AMP
0,FKCRRWQWRMAKLGA,0.998,0.002,0
1,PKIISSPLFKTLLSAVGSALSSSGGQE,0.777,0.223,0
2,TLKKSLLLIFFLGTISLSLC,0.847,0.153,0
3,NGVQPKYRWWRWWRRWWW,0.364,0.636,1
4,ITEVITILLNRLTDRLEK,0.756,0.244,0
...,...,...,...,...
337,GCKKYRRFRWKFKGKLFLFG,0.032,0.968,1
338,SAVWRHWRRFWLRKHRKH,0.237,0.763,1
339,FLFKLIPKAIKGLVKAIRK,0.288,0.712,1
340,GRLQAFLAKMKEIAAQTL,0.077,0.923,1


In [28]:
other_model = pickle.load(open(r'D:Colin/amp_paper/model/ET_gram_pos_feat_sel.sav', 'rb'))
calcamp.GetAllPred(df.Sequence, other_model)

C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator ExtraTreeClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator ExtraTreesClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  f"X has feature 

,Sequence,Proba_pred_non_amp,Proba_pred_amp,AMP
0,FKCRRWQWRMAKLGA,0.695,0.305,0
1,PKIISSPLFKTLLSAVGSALSSSGGQE,0.635,0.365,0
2,TLKKSLLLIFFLGTISLSLC,0.710,0.290,0
3,NGVQPKYRWWRWWRRWWW,0.670,0.330,0
4,ITEVITILLNRLTDRLEK,0.685,0.315,0
...,...,...,...,...
337,GCKKYRRFRWKFKGKLFLFG,0.710,0.290,0
338,SAVWRHWRRFWLRKHRKH,0.705,0.295,0
339,FLFKLIPKAIKGLVKAIRK,0.680,0.320,0
340,GRLQAFLAKMKEIAAQTL,0.710,0.290,0


## From CSV File

In [17]:
csv_file = r'D:Colin/amp_paper/dataset/dataset_ampep.csv'
df = calcamp.OpenCSV(csv_file).head(20)
df

,IDs,Sequence,Activity,Length
0,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,NYIYSGHNYHQ,0,11
1,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,DPNATIIMLGTGTGIAPFR,0,19
2,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,MGQGAVEGQLFYNVQ,0,15
3,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,MSQASSSPGEGPSSEAAAISEAEAASGS,0,28
4,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,INWKKIASIGKEVLKAL,0,17
5,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,MKKAWWKEGVVYQIY,0,15
6,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,AINSESGVRSVVPQPCNALPNQGPEK,0,26
7,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,NWRKILGKIAKVAAGLLGSMLAGYQV,0,26
8,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,YGEPIGVETLTK,0,12
9,unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_...,AGDTSSEAKGMWFGPRL,0,17


In [20]:
calcamp.GetAllPred(df.Sequence, et_tuned_g_pos)

C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


,Sequence,Proba_pred_non_amp,Proba_pred_amp,AMP
0,NYIYSGHNYHQ,0.770,0.230,0
1,DPNATIIMLGTGTGIAPFR,0.718,0.282,0
2,MGQGAVEGQLFYNVQ,0.758,0.242,0
3,MSQASSSPGEGPSSEAAAISEAEAASGS,0.930,0.070,0
4,INWKKIASIGKEVLKAL,0.632,0.368,0
5,MKKAWWKEGVVYQIY,0.611,0.389,0
6,AINSESGVRSVVPQPCNALPNQGPEK,0.821,0.179,0
7,NWRKILGKIAKVAAGLLGSMLAGYQV,0.046,0.954,1
8,YGEPIGVETLTK,0.745,0.255,0
9,AGDTSSEAKGMWFGPRL,0.850,0.150,0


## Simple predictions

In [24]:
calcamp.Pred_proba("ARARARA", et_tuned_g_pos) #either simple or list of sequences

C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


,Sequence,Proba_pred_non_amp,Proba_pred_amp
0,ARARARA,0.834,0.166


In [25]:
calcamp.Pred("ARARARA", et_tuned_g_pos) #either simple or list of sequences

C:\Users\bourn\Anaconda3\envs\CalcAMP\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


,Sequence,AMP
0,ARARARA,0
